In [1]:
import pandas as pd
df = pd.read_csv('ssbpts.csv')

df2 = df.query('MidLat > 30 and MidLat < 75 and MidLon < -60 and MidLon > -135 ')

In [5]:
try:
    dfe = pd.read_csv('errors.csv', index_col=0)
    df3 = pd.concat([df2, dfe], axis = 1)
    df4 = df3.query('Status != Status')
except FileNotFoundError:
    dfe = pd.DataFrame()
    df4 = df2
    


In [ ]:
from obspy import UTCDateTime

from obspy.taup import TauPyModel

from obspy.clients.fdsn import Client

client = Client("IRIS")

model = TauPyModel('ak135')



def save_error_dataframe(dataframe):
    dfe.to_csv('errors.csv')

for bp in df4.iterrows():
    etime = UTCDateTime(bp[1].EvtTime)
    sta = bp[1].Station
    net = bp[1].Network
    deldeg = bp[1].Range
    
    arrivals = model.get_travel_times(0.0, deldeg, phase_list = ['SS'])
    
    time_ss = arrivals[0].time
    
    t1 = etime + time_ss - 200
    t2 = etime + time_ss + 200
    
    try:
        st = client.get_waveforms(net, sta, "*", "LH*", t1, t2, attach_response = True)
    except:
        err = 'No Data at %s.%s' % (net, sta)
        print(err)
        dfe.loc[bp[0],"Status"] = err
        continue

    try:
        st.remove_response(output='ACC', pre_filt=(0.005, 0.01, 0.5, 1.0))
    except ValueError as e:
        err = 'Response Error: %s' % e
        print(err)
        dfe.loc[bp[0],"Status"] = err
    
    try:
        inv = client.get_stations(
            starttime=t1, endtime=t2, 
            network=net, sta=sta, 
            loc="*", channel="LH*", level="response")
        
        st.rotate('->ZNE', inventory = inv)
        st.rotate('NE->RT', back_azimuth = bp[1].BAzimuth)
    except ValueError as e:
        err = 'Rotation Error: %s' % e
        print(err)
        dfe.loc[bp[0],"Status"] = err
        
        continue
    
    bpid = '%012d' % bp[0]
    
    filename = 'data/mseed/' + bpid + '.mseed'
    
    trT = st[0]
    
    for tr in st:
        if 'T' in tr.stats.channel:
            trT = tr
    
    trT.resample(1)
    
    trT.write(filename)
    
    dfe.loc[bp[0],"Status"] = 'OK'
    
    save_error_dataframe(dfe)
   

/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warn

No Data at II.ALE


/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warn

No Data at AK.ANM
No Data at AK.ATKA
No Data at AK.BAGL
No Data at AK.BAL
No Data at AK.BARK
No Data at AK.BARN
No Data at AK.BCP
No Data at AK.BERG
No Data at AK.BESE
No Data at AK.BGLC
No Data at AK.BMR
No Data at AK.BPAW
No Data at AK.BRLK
No Data at AK.BRSE
No Data at AK.BWN
No Data at AK.CAST
No Data at AK.CCB
No Data at AK.CHI
No Data at AK.CHN
No Data at AK.CNP
No Data at AK.COLD
No Data at AK.CRQ
No Data at AK.CTG
No Data at AK.DCPH
No Data at AK.DHY
No Data at AK.DIV
No Data at AK.DOT
No Data at AK.EYAK
No Data at AK.FALS
No Data at AK.FID
No Data at AK.FIRE
No Data at AK.FYU
No Data at AK.GAMB
No Data at AK.GHO
No Data at AK.GLB
No Data at AK.GLI
No Data at AK.GOAT
No Data at AK.GRIN
No Data at AK.GRNC
No Data at AK.HDA
No Data at AK.HIN
No Data at AK.HMT
No Data at AK.HOM
No Data at AK.ISLE
No Data at AK.JIS
No Data at AK.KAI
No Data at AK.KHIT
No Data at AK.KIAG
No Data at AK.KLU
No Data at AK.KNK
No Data at AK.KTH
No Data at AK.KULT
No Data at AK.LOGN
No Data at AK.MCAR
No

/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warn

...

No Data at NU.ESPN
No Data at NU.ESTN
No Data at NU.HERN
No Data at NU.MASN


/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


No Data at ON.JAC01
No Data at ON.RSIS0
No Data at ON.RSIS1
No Data at ON.TER01
No Data at OV.BATAN
No Data at OV.CAO2
No Data at OV.CCOL
No Data at OV.CDITO
No Data at OV.CDM
No Data at OV.COVE
No Data at OV.CPMI
No Data at OV.DUNO
No Data at OV.HDC3
No Data at OV.HZTE
No Data at OV.JACO
No Data at OV.OCM
No Data at OV.ORTG
No Data at OV.PEZE
No Data at OV.PNPB
No Data at OV.POTG
No Data at OV.RIFO
No Data at OV.RIOS
No Data at OV.SRBA
No Data at OV.VMAR


/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warn

No Data at WI.ABD
No Data at WI.ANBD
Rotation Error: All components need to have the same time span.
No Data at WI.CBE
No Data at WI.DHS
No Data at WI.DSD
No Data at WI.DSLB
No Data at WI.GCMP
Rotation Error: All components need to have the same time span.
No Data at WI.MAGL
No Data at WI.MPOM
No Data at WI.SLBI
No Data at WI.TDBA
No Data at XY.PACH
No Data at XY.PAHC
No Data at XY.PAHR
